## Get Suspicious DNS

In [ ]:
import urllib2
import json
import os 
import datetime

# getting date from the parent path. 
path = os.getcwd().split("/") 
date = path[len(path)-1]   
dsource = path[len(path)-2]

In [ ]:
def apply_css_to_select(select):
    select._css = (
        (None, 'height', '90%'),
        (None, 'width', '90%'),
        ('select', 'overflow-x', 'auto'),
        ('select', 'width', '100%'),
        ('select', 'margin', 0)
    )

try:
    import ipywidgets as widgets # For jupyter/ipython >= 1.4
except ImportError:
    from IPython.html import widgets
from IPython.display import display, HTML, clear_output, Javascript 

def fill_list(list_control,source):
    options_list = ['- Select -'] 
    options_list.extend([s for s in source])
    list_control.options = options_list

# client panel
client_header = widgets.HTML(value="Client IP")
client_select = widgets.Select(height='90%')
apply_css_to_select(client_select)

client_box = widgets.Box(width='20%', height='100%')
client_box.children = [client_header, client_select]

# query panel
query_header = widgets.HTML(value="Query")
query_select = widgets.Select(height='90%')
apply_css_to_select(query_select)

query_box = widgets.Box(width='60%', height='100%')
query_box.children = [query_header, query_select]

# Actions Panel
actions_header = widgets.HTML(value="&nbsp;")
quick_text = widgets.Text(value='', width='100%', placeholder='Quick scoring')
quick_text._css = (
    (None, 'width', '100%'),
)
rating_btn = widgets.RadioButtons(description='Rating:', options=['1', '2', '3'], width='100%')
assign_btn = widgets.Button(description='Score', width='45%')
assign_btn.button_style = 'primary'
save_btn = widgets.Button(description='Save', width='45%')
save_btn.button_style = 'primary'
save_btn._css = (
    (None, 'margin-left', '10%'),
)
actions_box = widgets.Box(width='20%', height='100%')
actions_box.children = [actions_header,quick_text,rating_btn, assign_btn,save_btn]

scoring_form = widgets.HBox(width='90%', height=250)
scoring_form.children = [client_box,query_box,actions_box]

global score_values 

score_values = []

def data_loader(): 
    us_ips = []
    us_dns = []
    scored = []
    
    global score_values
    query="""query($date:SpotDateType!) {
            dns{
                suspicious(date:$date){
                dnsQuery
                clientIp
            }
        }
    }"""
    variables={
        'date': datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
    }
    
    for item in score_values:
        scored.append(item[0])
        scored.append(item[1])
    
    response = GraphQLClient.request(query, variables)
  
    for row in response['data']['dns']['suspicious']:           
        if row['clientIp'] not in us_ips and row['clientIp'] not in scored: 
            us_ips.append(row['clientIp'])
        if row['dnsQuery'] not in us_dns and row['dnsQuery'] not in scored:
            us_dns.append(row['dnsQuery'])  
            
    fill_list(client_select,us_ips)
    fill_list(query_select,us_dns)
    client_select.value = "- Select -"
    query_select.value = "- Select -"    


display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
data_loader()
display(scoring_form)

# Update Suspicious DNS

In [ ]:
import csv
import datetime
import subprocess 


def assign_score(b):
    clear_output()
    sev = int(rating_btn.selected_label) 

    if quick_text.value: 
        ip = ""
        dns = quick_text.value
        dns_sev = int(rating_btn.selected_label)  
        for query in query_select.options:
            if query.endswith(dns): 
                score_values.append((ip,query,sev))
    else: 
        ip = client_select.value if not "- Select -" in client_select.value else ""
        dns = query_select.value if not "- Select -" in query_select.value else ""
        score_values.append((ip,dns,sev))
        clear_output()
        
    if client_select.value != "- Select -":   
        display(Javascript("$(\"option:nth-of-type(0)[data-value='" + client_select.value +"']\").remove();"))
    if quick_text.value:
        display(Javascript("$(\"option:nth-of-type(1)[data-value$='" + dns +"']\").remove();"))
    elif query_select.value != "- Select -":
        display(Javascript("$(\"option:nth-of-type(1)[data-value='" + query_select.value +"']\").remove();"))

    client_select.value = "- Select -"
    query_select.value = "- Select -"
    quick_text.value = ""    
    
    data_loader() 
    print "Click the 'Save' button when you're finished scoring" 


def save(b):    
    variables=[]
    global score_values
    mutation="""mutation($input:[DnsScoreType!]!)
                {
                  dns{
                    score(input:$input)
                        {success}
                  }
                }""" 
    
    for row in score_values:
        variables.append({
            'date': datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d'),
            'clientIp': row[0] if row[0] != "" else None,
            'dnsQuery': row[1] if row[1] != "" else None,
            'score': row[2] if row[2] != "" else None 
            })

    var = {'input':variables}
    response = GraphQLClient.request(mutation,var)
    
    score_values = []
    if not 'errors' in response:
        clear_output()    
        display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
        data_loader() 
        display(scoring_form)
        display(Javascript('reloadParentData();')) 
        print "Suspicious connects successfully updated"
    else:
        print "An error ocurred: " + response['errors'][0]['message']
        
        
assign_btn.on_click(assign_score)
save_btn.on_click(save) 